In [5]:
# function that converts cupt file into bio labels
def parseme_cupt_to_bio(parseme_file):
    sentences = []
    tokens, labels = [], []
    active_mwes = set()  # keeps track of already started MWEs

    with open(parseme_file, encoding="utf-8") as f:
        for line in f:
            line = line.strip()

            if not line or line.startswith("#"):
                if tokens:
                    sentences.append({"tokens": tokens, "labels": labels})
                    tokens, labels, active_mwes = [], [], set()
                continue

            cols = line.split("\t")
            if len(cols) < 2:
                continue

            token = cols[1]
            mwe_col = cols[-1]

            label = "O"
            if mwe_col not in {"_", "*"}:
                for mwe in mwe_col.split(";"):
                    if ":" in mwe:
                        mwe_id, _ = mwe.split(":", 1)
                        if mwe_id not in active_mwes:
                            label = "B-MWE"
                            active_mwes.add(mwe_id)
                        else:
                            label = "I-MWE"
                    else:
                        if mwe in active_mwes:
                            label = "I-MWE"

            tokens.append(token)
            labels.append(label)

    return sentences

In [6]:
# checks that the function "parseme_cupt_to_bio" works properly
from collections import Counter

train_sents = parseme_cupt_to_bio("train.cupt")
all_labels = [lab for sent in train_sents for lab in sent["labels"]]
print(Counter(all_labels))

Counter({'O': 230002, 'I-MWE': 3237, 'B-MWE': 2200})


In [7]:
from datasets import Dataset, DatasetDict

# uploads files already converted by the function
train_sents = parseme_cupt_to_bio("train.cupt")

# creates a dataset
dataset = DatasetDict({
    "train": Dataset.from_list(train_sents)
})

print(dataset)

DatasetDict({
    train: Dataset({
        features: ['tokens', 'labels'],
        num_rows: 10641
    })
})


In [8]:
# filters train dataset with only sentences containing MWEs
def filter_with_mwe(sents):
    return [s for s in sents if any(l != "O" for l in s["labels"])]

train_sents_mwe = filter_with_mwe(train_sents)

print("Original train size:", len(train_sents))
print("Filtered train size:", len(train_sents_mwe))


Original train size: 10641
Filtered train size: 2169


In [9]:
from datasets import Dataset, DatasetDict

# splits data into train 80%, evaluation (dev) 10%, and test 10%
n = len(train_sents_mwe)
train_split = int(n * 0.8)
dev_split   = int(n * 0.9)

dataset = DatasetDict({
    "train": Dataset.from_list(train_sents_mwe[:train_split]),
    "dev": Dataset.from_list(train_sents_mwe[train_split:dev_split]),
    "test": Dataset.from_list(train_sents_mwe[dev_split:])
})

print(dataset)

DatasetDict({
    train: Dataset({
        features: ['tokens', 'labels'],
        num_rows: 1735
    })
    dev: Dataset({
        features: ['tokens', 'labels'],
        num_rows: 217
    })
    test: Dataset({
        features: ['tokens', 'labels'],
        num_rows: 217
    })
})


In [10]:
# maps each label to a numerical id for later use
from collections import Counter

# list of labels in the dataset
unique_labels = sorted(set(lab for sent in train_sents for lab in sent["labels"]))
label2id = {label: i for i, label in enumerate(unique_labels)}
id2label = {i: label for label, i in label2id.items()}

print("Label2ID:", label2id)
print("ID2Label:", id2label)

def convert_labels_to_ids(examples):
    new_labels = []
    for lbls in examples["labels"]:
        new_labels.append([label2id[l] for l in lbls])
    examples["labels"] = new_labels
    return examples

dataset = dataset.map(convert_labels_to_ids, batched=True)

# we tokenize and align with subwords
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("dbmdz/bert-base-italian-cased")

def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples["tokens"],
        is_split_into_words=True,
        truncation=True,
        padding="max_length",
        max_length=128,
    )

    all_labels = []
    for i, labels in enumerate(examples["labels"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)  # ignorato dal loss
            elif word_idx != previous_word_idx:
                label_ids.append(labels[word_idx])
            else:
                if labels[word_idx] == label2id["B-MWE"]:
                    label_ids.append(label2id["I-MWE"])
                else:
                    label_ids.append(labels[word_idx])
            previous_word_idx = word_idx
        all_labels.append(label_ids)

    tokenized_inputs["labels"] = all_labels
    return tokenized_inputs

tokenized_dataset = dataset.map(tokenize_and_align_labels, batched=True)
print(tokenized_dataset)

Label2ID: {'B-MWE': 0, 'I-MWE': 1, 'O': 2}
ID2Label: {0: 'B-MWE', 1: 'I-MWE', 2: 'O'}


Map:   0%|          | 0/1735 [00:00<?, ? examples/s]

Map:   0%|          | 0/217 [00:00<?, ? examples/s]

Map:   0%|          | 0/217 [00:00<?, ? examples/s]

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


...

vocab.txt: 0.00B [00:00, ?B/s]

Map:   0%|          | 0/1735 [00:00<?, ? examples/s]

Map:   0%|          | 0/217 [00:00<?, ? examples/s]

Map:   0%|          | 0/217 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['tokens', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1735
    })
    dev: Dataset({
        features: ['tokens', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 217
    })
    test: Dataset({
        features: ['tokens', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 217
    })
})


In [11]:
!pip install seqeval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16162 sha256=74db4e3225cfab8fd3224e01d5df2a67bc57582f2936bd5ce698e4650f96c068
  Stored in directory: /root/.cache/pip/wheels/5f/b8/73/0b2c1a76b701a677653dd79ece07cfabd7457989dbfbdcd8d7
Successfully built seqeval


In [12]:
import torch
import numpy as np
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer
from seqeval.metrics import classification_report, f1_score

# check GPU: the model should run on GPU
print("GPU available:", torch.cuda.is_available())

GPU available: True


In [13]:
# creates the model from a pre-trained bert model
num_labels = len(label2id)

model = AutoModelForTokenClassification.from_pretrained(
    "dbmdz/bert-base-italian-cased",
    num_labels=num_labels,
    id2label=id2label,
    label2id=label2id
)

model.safetensors:   0%|          | 0.00/442M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at dbmdz/bert-base-italian-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [14]:
# set preferred training variables
training_args = TrainingArguments(
    output_dir="./mwe_bert",
    learning_rate=3e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=5,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=20,
    save_total_limit=1,
)


In [15]:
# set the evaluation of the model with true labels, true predictions and f1 score
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_labels = [
        [id2label[l] for (l, lab) in zip(label_row, label_row_orig) if lab != -100]
        for label_row, label_row_orig in zip(predictions, labels)
    ]
    true_preds = [
        [id2label[pred] for (pred, lab) in zip(pred_row, label_row) if lab != -100]
        for pred_row, label_row in zip(predictions, labels)
    ]

    f1 = f1_score(true_labels, true_preds)
    report = classification_report(true_labels, true_preds)
    return {"f1": f1, "report": report}

In [16]:
# set the trainer datasets and the other necessary variables
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["dev"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

/tmp/ipython-input-2720213067.py:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [17]:
# trains the model
# if needed, API key is: e84cd7f639d3bf50b4a74761d5ea1fde28a455af
trainer.train()

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: dueblanda (dueblanda-universit-tsstadt-t-bingen) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss
20,0.425500
40,0.205400
60,0.148300
80,0.113200
100,0.106900
120,0.093100
140,0.073300
160,0.077600
180,0.061800
200,0.060800


TrainOutput(global_step=275, training_loss=0.11304987907409668, metrics={'train_runtime': 197.3287, 'train_samples_per_second': 43.962, 'train_steps_per_second': 1.394, 'total_flos': 566692464556800.0, 'train_loss': 0.11304987907409668, 'epoch': 5.0})

In [18]:
# checks statistics about performance
metrics = trainer.evaluate()
print(metrics)

{'eval_loss': 0.1065702885389328, 'eval_f1': 1.0, 'eval_report': '              precision    recall  f1-score   support\n\n         MWE       1.00      1.00      1.00       404\n\n   micro avg       1.00      1.00      1.00       404\n   macro avg       1.00      1.00      1.00       404\nweighted avg       1.00      1.00      1.00       404\n', 'eval_runtime': 1.5601, 'eval_samples_per_second': 139.089, 'eval_steps_per_second': 4.487, 'epoch': 5.0}


In [19]:
# saves the trained model and tokenizer
trainer.save_model("./mwe_bert")
tokenizer.save_pretrained("./mwe_bert")

print("Model and tokenizer saved in ./mwe_bert")

Model and tokenizer saved in ./mwe_bert


In [20]:
from transformers import pipeline

# loads the trained model
inference_pipeline = pipeline(
    "token-classification",
    model="./mwe_bert",
    tokenizer=tokenizer,
    aggregation_strategy="simple"
)

# takes the first three sentences from the evaluation dataset to test
for i in range(3):
    example = tokenized_dataset["dev"][i]
    tokens = tokenizer.convert_ids_to_tokens(example["input_ids"], skip_special_tokens=True)

    true_labels = [id2label[label] for label in example["labels"] if label != -100]

    sentence = tokenizer.decode(example["input_ids"], skip_special_tokens=True)

    # model predictions
    preds = inference_pipeline(sentence)

    print(f"\n--- Example {i+1} ---")
    print("Sentence:", sentence)
    print("True labels:", true_labels)
    print("Predictions:", preds)

Device set to use cuda:0



--- Example 1 ---
Sentence: La revoca dell'di l'incarico all'a l'assessore leghista ha portato alla a la rottura tra il gruppo leghista e il resto della di la maggioranza.
True labels: ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-MWE', 'O', 'I-MWE', 'O', 'I-MWE', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
Predictions: [{'entity_group': 'MWE', 'score': np.float32(0.97959423), 'word': 'portato', 'start': 59, 'end': 66}, {'entity_group': 'MWE', 'score': np.float32(0.9893167), 'word': 'a', 'start': 72, 'end': 73}]

--- Example 2 ---
Sentence: In corsa, Räikkönen scatta al a il comando ma deve ritirar si per un guasto tecnico dopo pochi giri, così come Rubens Barrichello con la Ferrari.
True labels: ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-MWE', 'I-MWE', 'I-MWE', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
Predic

In [21]:
# prediction outputs
pred_output = trainer.predict(tokenized_dataset["test"])
logits = pred_output.predictions
labels = pred_output.label_ids

pred_ids = logits.argmax(axis=-1)

y_true = []
y_pred = []
for lab_row, pred_row in zip(labels, pred_ids):
    true_row = []
    pred_row_labels = []
    for l, p in zip(lab_row, pred_row):
        if l == -100:
            continue
        true_row.append(id2label[int(l)])
        pred_row_labels.append(id2label[int(p)])
    y_true.append(true_row)
    y_pred.append(pred_row_labels)

# seqeval evaluation metrics
from seqeval.metrics import classification_report, f1_score, precision_score, recall_score
print("F1 (micro-like) :", f1_score(y_true, y_pred))
print(classification_report(y_true, y_pred))


F1 (micro-like) : 0.5870646766169155
              precision    recall  f1-score   support

         MWE       0.56      0.62      0.59       381

   micro avg       0.56      0.62      0.59       381
   macro avg       0.56      0.62      0.59       381
weighted avg       0.56      0.62      0.59       381



In [22]:
def bio_spans(labels):
    """labels: list of BIO strings per token -> return list of (start, end) spans"""
    spans = []
    start = None
    for i, lab in enumerate(labels):
        if lab.startswith("B-"):
            if start is not None:
                spans.append((start, i-1))
            start = i
        elif lab.startswith("I-"):
            continue
        else:  # 'O'
            if start is not None:
                spans.append((start, i-1))
                start = None
    if start is not None:
        spans.append((start, len(labels)-1))
    return spans

# compute precision, recall and f1 score
tp = 0
pred_count = 0
gold_count = 0
for true_row, pred_row in zip(y_true, y_pred):
    gold_spans = set(bio_spans(true_row))
    pred_spans = set(bio_spans(pred_row))
    tp += len(gold_spans & pred_spans)
    pred_count += len(pred_spans)
    gold_count += len(gold_spans)

precision = tp / pred_count if pred_count else 0.0
recall = tp / gold_count if gold_count else 0.0
f1 = (2 * precision * recall / (precision + recall)) if (precision+recall)>0 else 0.0
print(f"Span-level P={precision:.4f} R={recall:.4f} F1={f1:.4f}")


Span-level P=0.5763 R=0.6343 F1=0.6039


In [23]:
# identifies and saves mismatches in the datasets
import csv
out = []
for i, (example, true_row, pred_row) in enumerate(zip(tokenized_dataset["test"], y_true, y_pred)):
    if true_row != pred_row:
        sent = tokenizer.decode(example["input_ids"], skip_special_tokens=True)
        out.append({
            "idx": i,
            "sentence": sent,
            "true": " ".join(true_row),
            "pred": " ".join(pred_row)
        })

keys = ["idx","sentence","true","pred"]
with open("mwe_mismatches.csv","w",encoding="utf-8", newline="") as f:
    writer = csv.DictWriter(f, fieldnames=keys)
    writer.writeheader()
    writer.writerows(out)

print(f"Saved {len(out)} mismatches to mwe_mismatches.csv")


Saved 135 mismatches to mwe_mismatches.csv


In [24]:
# tests the model with random sentences inserted manually
from transformers import pipeline

nlp = pipeline(
    "token-classification",
    model="./mwe_bert",
    tokenizer="./mwe_bert",
    aggregation_strategy="simple"
)

# write any sentence here
sentences = [
    "Mettiamo in pratica un piano ben congeniato per risolvere il problema.",
    "Ha preso una decisione di comune accordo con i colleghi.",
    "Mi vado a fare la doccia.",
    "Il mio modello funziona!",
    "Dimmi una frase qualunque"
]

for sentence in sentences:
    print(f"Sentence: {sentence}")
    predictions = nlp(sentence)
    for pred in predictions:
        print(f"  MWE: {pred['word']}, Score: {pred['score']:.2f}, Start: {pred['start']}, End: {pred['end']}")
    print()


Device set to use cuda:0


Sentence: Mettiamo in pratica un piano ben congeniato per risolvere il problema.
  MWE: Mettiamo in pratica, Score: 0.99, Start: 0, End: 19

Sentence: Ha preso una decisione di comune accordo con i colleghi.
  MWE: preso una decisione, Score: 0.91, Start: 3, End: 22
  MWE: accordo, Score: 0.66, Start: 33, End: 40

Sentence: Mi vado a fare la doccia.
  MWE: vado a fare la doccia, Score: 0.82, Start: 3, End: 24

Sentence: Il mio modello funziona!

Sentence: Dimmi una frase qualunque
  MWE: Dimmi una frase, Score: 0.91, Start: 0, End: 15

